In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
import matplotlib.pyplot as plt
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback

import prettytable

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
path = os.getcwd()
path

# Pre Processing

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

# Removing columns based on dependent column


In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

# Reshaping the data

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0],1, x_train.shape[1]))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],1, x_test.shape[1]))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

# Sigmoid Funtion

In [ ]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

# Keras Model Classifier

In [ ]:
def keras_model():
    activation='relu'
    dropout_rate=0.2
    init_mode='uniform'
    weight_constraint=0 
    optimizer='adam' 
    lr = 0.01
    momemntum=0
    
    model = Sequential()
    model.add(Dense(units = 8, input_dim = 75, kernel_initializer=init_mode,activation=activation))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Parameters Tuning

In [ ]:
def tuning(model,epochs,batch_size,X_train,Y_train):
    
    param_grid = dict(epochs=epochs, batch_size=batch_size)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train,Y_train,verbose = 0) 
    epoch = grid_result.best_params_['epochs']
    batch_size = grid_result.best_params_['batch_size']
    
    return (epoch,batch_size)

# Recurrent Neural Networks LSTM

In [ ]:
def recurrent_neural_networks(X,Y,layers,epochs = 5,batch_size = 32,units=50):
    print(batch_size,epochs)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    x_train, x_test, y_train, y_test = reshape_data(x_train, x_test, y_train, y_test)

    model = build_lstm(x_train,layers,units)
    model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    
    history_lstm = model.fit(x_train,y_train,epochs = epochs,batch_size = batch_size, validation_data = (x_test,y_test),shuffle = False,verbose=0)

    y_pred = model.predict(x_test)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],1))

    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    c = 0
    for a,b in zip(y_pred,y_test):
        if a * b >= 0:
            c += 1
    direction = c/len(y_test)
    try:
        table.add_row([units,epochs,batch_size,layers,direction,rmse,mae,mse,r2])
    except :
        pass
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2,"same direction":c,"total":len(y_test)}


# LSTM Model

In [ ]:
def build_lstm(X_train,layers,units = 50):
    model = Sequential()
    model.add(LSTM(units=units,return_sequences=True,input_shape=(1,X_train.shape[2])))
    model.add(Dropout(0.2))
    for i in range(layers):s
        model.add(LSTM(units=units,return_sequences=True))
        model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'swish'))
    return model

In [ ]:
combinations = list()
def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
u = [50,100]
b = [25,32,48,64]
e = [25,50,75,100]
arr = [u,b,e]
get_combinations(arr)

In [ ]:
len(combinations)

In [ ]:
units = 50
# batch_size = 25
# epochs = 100
layers = 3
top = pd.read_csv("top.csv")

In [ ]:
%%time
tables = list()
columns = ["units","epochs","batch_size","layers","direction","root_mean_squared_error","mean_absolute_error","mean_squared_error","rsquared_adj"]
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if filename.startswith("gr500680"):
        try:
            table = prettytable.PrettyTable()
            stock_name = top[top["security id"] == int(filename[2:8])]["name"].values[0]
            table.field_names = ["units","epochs","batch_size","layers","direction","root_mean_squared_error","mean_absolute_error","mean_squared_error","rsquared_adj"]
            table.title = str(filename[2:8]) + " " + stock_name + " RNN"
            print(table.title)
            df = pd.read_csv(os.path.join(path,"Data\Stock\\"+filename))
            column = "Next Day Close Price GR"
            df = pre_process_data(df,60)
            (data,column) = dependent_column(df,column)
            X = data[data.columns[:-1]]
            Y = data[column]
            arguments = list()
            for batch_size,epochs in combinations:
                data = [X,Y,layers,epochs,batch_size,units]
                arguments.append(data)
            threads = ThreadPool(multiprocessing.cpu_count())
            result = threads.starmap(recurrent_neural_networks,arguments)
            # X,Y,layers,epochs,batch_size,units = arguments[0]
            recurrent_neural_networks(X,Y,layers,epochs,batch_size,units)

            tables.append(table)
        except Exception as e :
            traceback.print_exc()
            pass

# rnn.to_csv(os.path.join(path,"Results\\"+"rnn"+".csv"),index=None)

In [61]:
s = '''+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                    500680 PFIZER LTD RNN                                                                     |
+-------+--------+------------+--------+---------------------+-------------------------+----------------------+------------------------+-----------------------+
| units | epochs | batch_size | layers |      direction      | root_mean_squared_error | mean_absolute_error  |   mean_squared_error   |      rsquared_adj     |
+-------+--------+------------+--------+---------------------+-------------------------+----------------------+------------------------+-----------------------+
|   50  |   25   |     25     |   3    | 0.48522727272727273 |   0.01756240545186482   | 0.012506045704708032 | 0.00030843808525569114 | -0.051527386056182234 |
|   50  |   25   |     16     |   3    |  0.4863636363636364 |   0.017539184423609765  | 0.012452760844727902 | 0.00030762299024539536 |  -0.04874856344465783 |
|   50  |   50   |     25     |   3    |  0.4909090909090909 |   0.017648071174570377  | 0.012576733349421039 | 0.0003114544161827018  |  -0.06181066405192848 |
|   50  |   75   |     25     |   3    | 0.48863636363636365 |   0.017728282869037704  | 0.01271896263516963  | 0.00031429201348461575 |  -0.07148460321896777 |
|   50  |   50   |     16     |   3    |        0.4875       |   0.017914617603431902  | 0.012888297156434721 | 0.00032093352387719214 |  -0.0941268461727891  |
|   50  |  100   |     25     |   3    |  0.4943181818181818 |   0.017685173325642442  | 0.01267810516521828  | 0.00031276535555801495 |  -0.06627991969997327 |
|   50  |   75   |     16     |   3    | 0.48295454545454547 |   0.017827517927239838  | 0.012777746838865255 | 0.00031782039544605775 |  -0.08351356604256721 |
|   50  |   25   |     32     |   3    |        0.4875       |   0.017458741128591717  | 0.012338371303608736 | 0.00030480764179518003 |  -0.03915047508200287 |
|   50  |  100   |     16     |   3    |  0.4818181818181818 |   0.01776131660890784   | 0.012651664195248507 | 0.00031546436768186553 |  -0.07548139415853061 |
|   50  |   50   |     32     |   3    | 0.49204545454545456 |   0.017483380940780537  | 0.012302706088646026 | 0.0003056686091204481  |  -0.04208568562926196 |
|   50  |   25   |     48     |   3    |        0.4875       |   0.01739462976405626   | 0.012265390071909863 | 0.0003025731446285919  |  -0.03153262541604951 |
|   50  |   75   |     32     |   3    | 0.48409090909090907 |   0.017602004408079904  | 0.012412824471386811 | 0.00030983055918206435 |  -0.05627460936596518 |
|   50  |   50   |     48     |   3    |  0.4909090909090909 |   0.01746889789916706   | 0.012300893929883924 | 0.0003051623938115233  |  -0.04035989596183498 |
|   50  |   25   |     64     |   3    |  0.4818181818181818 |   0.01732659752324078   |  0.0120834345476863  | 0.00030021098173237354 | -0.023479537634622405 |
|   50  |   75   |     48     |   3    |  0.4965909090909091 |   0.017461110611882387  | 0.012222056118026221 | 0.0003048903838003917  |  -0.0394325591974749  |
|   50  |  100   |     32     |   3    | 0.49318181818181817 |   0.017533967125098728  | 0.012381388826215645 |  0.000307440003144043  |  -0.04812472366103471 |
|   50  |   50   |     64     |   3    |  0.4784090909090909 |   0.017386096885174618  | 0.012119925755687029 | 0.0003022763649006785  | -0.030520843711891565 |
|   50  |  100   |     48     |   3    |  0.4909090909090909 |   0.017486485494152538  | 0.012270690561579001 | 0.0003057771749372071  |  -0.04245580830531548 |
|   50  |   75   |     64     |   3    |  0.4954545454545455 |   0.017451709445867245  | 0.01213023711421472  |  0.000304562162582972  |  -0.03831358714019695 |
|   50  |  100   |     64     |   3    | 0.49886363636363634 |   0.01757609616271123   | 0.012285508079620032 | 0.00030891915632087237 | -0.053167453946170395 |
|   50  |  100   |     64     |   3    | 0.48863636363636365 |   0.017471663968224325  | 0.012188481467001062 | 0.0003052590418185482  |  -0.04068938843067249 |
+-------+--------+------------+--------+---------------------+-------------------------+----------------------+------------------------+-----------------------+
'''

In [62]:
result = [tuple(filter(None, map(str.strip, splitline))) for line in s.splitlines() for splitline in [line.split("|")] if len(splitline) > 1]

In [63]:
resultdf = pd.DataFrame(result[2:],columns=result[1])
resultdf = resultdf.sort_values(by=['direction'],ascending=[False])
resultdf

,units,epochs,batch_size,layers,direction,root_mean_squared_error,mean_absolute_error,mean_squared_error,rsquared_adj
19,50,100,64,3,0.49886363636363634,0.01757609616271123,0.012285508079620032,0.00030891915632087237,-0.053167453946170395
14,50,75,48,3,0.4965909090909091,0.017461110611882387,0.012222056118026221,0.0003048903838003917,-0.0394325591974749
18,50,75,64,3,0.4954545454545455,0.017451709445867245,0.01213023711421472,0.000304562162582972,-0.03831358714019695
5,50,100,25,3,0.4943181818181818,0.017685173325642442,0.01267810516521828,0.00031276535555801495,-0.06627991969997327
15,50,100,32,3,0.49318181818181817,0.017533967125098728,0.012381388826215645,0.000307440003144043,-0.04812472366103471
9,50,50,32,3,0.49204545454545456,0.017483380940780537,0.012302706088646026,0.0003056686091204481,-0.04208568562926196
12,50,50,48,3,0.4909090909090909,0.01746889789916706,0.012300893929883924,0.0003051623938115233,-0.04035989596183498
2,50,50,25,3,0.4909090909090909,0.017648071174570377,0.012576733349421039,0.0003114544161827018,-0.06181066405192848
17,50,100,48,3,0.4909090909090909,0.017486485494152538,0.012270690561579001,0.0003057771749372071,-0.04245580830531548
20,50,100,64,3,0.48863636363636365,0.017471663968224325,0.012188481467001062,0.0003052590418185482,-0.04068938843067249


In [ ]:
mydf = pd.read_csv('F:\PracticumProject\StockAnalysisTool\Data\Stock\gr500680.csv')
column = "Next Day Close Price GR"
mydf = pre_process_data(mydf,60)
(data,column) = dependent_column(mydf,column)
X = data[data.columns[:-1]]
Y = data[column]

In [ ]:
help(recurrent_neural_networks)

In [ ]:
newres = recurrent_neural_networks(X,Y,3,100,32,100)

In [ ]:
newres

In [ ]:
pt = prettytable.PrettyTable()
pt.field_names = list(newres.keys())
pt.add_row(list(newres.values()))

In [58]:
s= pt.get_string()

In [59]:
r = [tuple(filter(None, map(str.strip, splitline))) for line in s.splitlines() for splitline in [line.split("|")] if len(splitline) > 1]
r

[('root_mean_squared_error',
  'mean_absolute_error',
  'mean_squared_error',
  'rsquared_adj',
  'same direction',
  'total'),
 ('0.017862463256434726',
  '0.012774004423953404',
  '0.0003190675935874807',
  '-0.08776551502110341',
  '421',
  '880')]

In [60]:
rdf = pd.DataFrame(r[1:],columns=r[0])
# rdf = rdf.sort_values(by=['direction'],ascending=[False])
rdf

,root_mean_squared_error,mean_absolute_error,mean_squared_error,rsquared_adj,same direction,total
0,0.017862463256434726,0.012774004423953404,0.0003190675935874807,-0.08776551502110341,421,880
